In [ ]:
import sys
import pathlib
#sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))
#sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().joinpath('Quantum_Annealing_for_Particle_Matching'))))
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd())))
for p in sys.path:
    print(p)
import os
import time
import h5py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import gurobipy as gp

import dwave
import dwave.system
from src.particle_funcs import distance_matrix as distance_matrix
import src.leap_funcs.qubo.q_matrix as q_matrix
from src.leap_funcs import embedding_quality
from src.leap_funcs.qubo import filter_samples
from src._misc import compare_matrices

from src import h5py_funcs
from src.h5py_funcs import discoveries, init_custom_getstates, io, parameterstudy_using_info_file